In [ ]:
# import libraries and navigate to directory
%cd '/home/uzumochi/eigenjuno/data/raw/perijove_27'
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from PIL import Image
import json, cv2, warnings, os, threading, time, multiprocessing
from util import *
from framelets import *
from jitter_correction import *
from warnings import filterwarnings
filterwarnings('ignore')

In [ ]:
# stitch function for thread, taken largely from the reddit post linked in repo desc
# one change i would like to make is centering the stitching around the actual planet
# (in some outputs most of the stitched image will be just black space if image not large enough)
def stitch(meta_num):
    im_info = meta_num + '-Metadata.json'
    with open(im_info, 'r') as json_file:
        im_info_dir = json.load(json_file)
        image = im_info_dir['FILE_NAME']
    
#     img = mpimg.imread(image)
    img = Image.open(image)
    im_ar = np.array(img)
#     print(im_ar.shape[0])
#     plt.imshow(img)
    
#     import cv2
#     from scipy.ndimage import label
#     im_crop = np.zeros([1, im_ar.shape[1]])
#     for row in range(im_ar.shape[0]):
#         arr = im_ar[row, :]
#         print(arr.mean())
#         if arr.mean() > 0.01: # evaluates to true if the row isnt close to completely black
#             print('not zero')
#             im_crop = np.vstack([im_crop,arr])
#             np.append(im_crop, arr)
#             im_crop.append(arr)
#     print(im_ar.shape)
#     print(im_crop.shape)
#     plt.imshow(im_crop)

#     im = cv2.imread(image)
#     l, _ = label(im)
#     mask = (l != 0)
#     print(mask)
#     for row in range(im.shape[0]):
#         if np.any(im[row, :]):
#             print(row)
#     out = im[np.ix_(np.any(mask, axis = 1), np.any(mask, axis = 0))]
#     plt.imshow(out)
    
#     print(img.shape[1])
    
#     # crop down to edges    
#     img_crop = np.array([])
#     for r in range(img.shape[0]):
#         num = 0
#         for c in range(img.shape[1]):
#             if img[r, c] == 0.0:
#                 num += 1
#         if num < img.shape[1]:
#             print(num)
#             np.append(img_crop, img[r, :])

#     plt.imshow(img_crop)
#     img = img_crop
#     print(img_crop.shape)
    
#     img_crop = np.array([])
#     for c in range(img.shape[1]):
#         num = 0
#         for r in range(img.shape[0]):
#             if img_crop[r, c] == 0.0:
#                 num += 1
#         if num < img.shape[0]:
#             np.append(img_crop, img[:, c], axis=1)
    
#     img = img_crop
#     plt.imshow(img)
    
    s1, s2 = im_ar.shape

    start_time = im_info_dir["START_TIME"]
    frame_delay = float(im_info_dir["INTERFRAME_DELAY"].split()[0])+0.001
    
    start_correction, frame_delay = correct_image_start_time_and_frame_delay(im_ar, start_time, frame_delay)
    
    framelets = generate_framelets(im_ar, start_time, start_correction, frame_delay)
    # removed color correction - im_ar.astype(np.int) ** 2

    cam_pos, cam_orient = get_junocam_jupiter_rel_pos_orient(start_time, start_correction + 17 * frame_delay)

    y, x = np.mgrid[-1024:1024,-1024:1024]
    x += 300
    y += 50
    rays = np.concatenate([x[...,None], y[...,None], np.ones(((2048),(2048),1))*fl[0]], axis=-1)
    rays = rays.dot(cam_orient)

    surface_raster, _ = project_onto_jupiter_surf(cam_pos, rays)

    colors = np.zeros(((2048),(2048),3))
    color_counts = np.zeros(((2048),(2048),3))

    for k,framelet in enumerate(framelets):
        print(str(meta_num) + ': processing framelet {} of {}..'.format(k+1, len(framelets)))
        col = framelet.color
#         print(type(surface_raster))
        brightnesses, valid_map = framelet.get_pixel_val_at_surf_point(surface_raster) # this is where it slows down
        colors[...,2-col] += brightnesses
        color_counts[...,2-col] += valid_map

    colors /= np.maximum(color_counts, 1)
    colors *= 255 / np.max(colors)

    colors = colors.astype(np.uint8)

    new_img = Image.fromarray(colors)
    new_img.save("out_" + str(meta_num) + ".png")

In [ ]:
#begin multithreaded execution - this will process all images in a directory, assuming they are from the same perijove
img_nums = []
for files in os.listdir('/home/uzumochi/eigenjuno/data/raw/perijove_27'):
    if files.endswith('.json'):
        img_nums.append(files[0 : 4])
print_str = 'Processing the following images: '
for im in img_nums:
    print_str += (im + ', ')
print_str = print_str[:-2]
print(print_str)

num_cpus = multiprocessing.cpu_count()
curr = 0

while curr < len(img_nums):
    threads = []
    num_threads = num_cpus if num_cpus < len(img_nums) else len(img_nums)
#     print(num_threads)
    for t in range(num_threads):
        threads.append(threading.Thread(target = stitch, args = (str(img_nums[curr]), )))
#         print(t)
        threads[t].start()
        curr += 1
    for t in threads:
        t.join()

print("done! :D")